In [1]:
from eppy import modeleditor
from eppy.modeleditor import IDF
from matplotlib import pyplot as plt
import os
import pandas as pd
import besos as besos
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem

#### Define the IDF, IDD and the weather file

In [2]:
idd_file='/usr/local/EnergyPlus-9-0-1/Energy+.idd'
weather_file ='/home/ict4bd/Downloads/epw/RUS_Moscow.276120_IWEC.epw'

In [3]:
building = ef.get_building('/home/ict4bd/Downloads/idfs/V9/on_off.idf')

In [4]:
insulation = FieldSelector(class_name='Material', object_name='Cement/plaster/mortar - gypsum plaster_O.OO2', field_name='Thickness')
insulationPR = Parameter(selector=insulation, value_descriptor=CategoryParameter(options=[0.0001,0.05,0.2]),name='Insulation Thickness')
window_to_wall = wwr(RangeParameter(0.1,0.9))

parameters = [window_to_wall, insulationPR]

#### setting optimal found values as sample

In [5]:
samples_temp =[]
samples_temp.append({'Window to Wall Ratio': 0.1,                     
                     'Insulation Thickness': 0.2,})
samples = pd.DataFrame.from_dict(samples_temp)
samples

,Window to Wall Ratio,Insulation Thickness
0,0.1,0.2


In [6]:
objectives = ['Electricity:Facility', 'DistrictHeating:Facility', 'DistrictCooling:Facility']

problem = EPProblem(parameters, objectives)
evaluator = EvaluatorEP(problem, building, out_dir='Output/output_besos_best_parameters1',
                        error_mode='Silent', err_dir='Output/output_besos_best_parameters1',
                        epw= weather_file)
outputs = evaluator.df_apply(samples, keep_input=True)    
outputs

,Window to Wall Ratio,Insulation Thickness,Electricity:Facility,DistrictHeating:Facility,DistrictCooling:Facility
0,0.1,0.2,1.278792e+11,8.151271e+10,5.576173e+09


In [7]:
idf_file = '/home/ict4bd/Downloads/NA/Output/output_besos_best_parameters2/BESOS_Output/in.idf'
IDF.setiddname(idd_file)
idf = IDF(idf_file,weather_file)

#### Performing simulation

In [8]:
idf.run(readvars=True,output_directory=u'Output/output_EPPY')


/usr/local/EnergyPlus-9-0-1/energyplus --weather /home/ict4bd/Downloads/epw/RUS_Moscow.276120_IWEC.epw --output-directory /home/ict4bd/Downloads/NA/Output/output_EPPY2 --idd /usr/local/EnergyPlus-9-0-1/Energy+.idd --readvars /home/ict4bd/Downloads/NA/in.idf



#### Performing file management to extract desired results

In [9]:
csv_columns = ['Date/Time',
              'BLOCK1:ZONE2:Zone Operative Temperature [C](Hourly)',
              'BLOCK1:ZONE1:Zone Operative Temperature [C](Hourly)',
              'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)',
              'DistrictHeating:Facility [J](Hourly)',
              'DistrictCooling:Facility [J](Hourly)',
              'Electricity:Facility [J](Hourly)']
              #'InteriorEquipment:Electricity [J](Hourly)']

In [10]:
df_columns = {'Date/Time':'Date_Time',
              'BLOCK1:ZONE2:Zone Operative Temperature [C](Hourly)':'t_in_ZONE2',
              'BLOCK1:ZONE1:Zone Operative Temperature [C](Hourly)':'t_in_ZONE1',
              'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)':'t_out',
              'DistrictHeating:Facility [J](Hourly)':'power_heating',
              'DistrictCooling:Facility [J](Hourly)':'power_cooling' ,
              'Electricity:Facility [J](Hourly)':'power_electricity'}
              #'InteriorEquipment:Electricity [J](Hourly)':'power_InteriorEquipment'}

In [12]:
idf_data =pd.read_csv('/home/ict4bd/Downloads/NA/Output/output_EPPY/eplusout.csv',usecols=csv_columns)

In [13]:
idf_data = idf_data.rename(columns =df_columns)

In [14]:
idf_data['t_in'] = idf_data[['t_in_ZONE2','t_in_ZONE1']].mean(axis=1)
idf_data.drop(['t_in_ZONE2', 't_in_ZONE1'], axis = 1)

,Date_Time,t_out,power_electricity,power_heating,power_cooling,t_in
0,01/01 00:15:00,NaN,NaN,NaN,NaN,NaN
1,01/01 00:30:00,NaN,NaN,NaN,NaN,NaN
2,01/01 00:45:00,NaN,NaN,NaN,NaN,NaN
3,01/01 01:00:00,-7.4500,710174.145981,2.630603e+07,0.0,11.598198
4,01/01 01:15:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
35035,12/31 23:00:00,-5.1750,710174.145981,1.964846e+07,0.0,11.975516
35036,12/31 23:15:00,NaN,NaN,NaN,NaN,NaN
35037,12/31 23:30:00,NaN,NaN,NaN,NaN,NaN
35038,12/31 23:45:00,NaN,NaN,NaN,NaN,NaN


In [15]:
idf_data = idf_data.dropna()

In [16]:
idf_data.reset_index(drop = True)

,Date_Time,t_out,t_in_ZONE2,t_in_ZONE1,power_electricity,power_heating,power_cooling,t_in
0,01/01 01:00:00,-7.4500,11.529470,11.666926,710174.145981,2.630603e+07,0.0,11.598198
1,01/01 02:00:00,-7.5625,11.529724,11.665294,710174.145981,2.649795e+07,0.0,11.597509
2,01/01 03:00:00,-8.4625,11.499248,11.663070,710174.145981,2.746139e+07,0.0,11.581159
3,01/01 04:00:00,-9.1750,11.410642,11.660477,710174.145981,2.814473e+07,0.0,11.535560
4,01/01 05:00:00,-9.7125,11.319871,11.657485,710174.145981,2.868359e+07,0.0,11.488678
...,...,...,...,...,...,...,...,...
8755,12/31 20:00:00,-4.0875,11.958043,12.063459,710174.145981,1.802262e+07,0.0,12.010751
8756,12/31 21:00:00,-4.3875,11.946286,12.052794,710174.145981,1.848567e+07,0.0,11.999540
8757,12/31 22:00:00,-4.6875,11.934234,12.041492,710174.145981,1.894636e+07,0.0,11.987863
8758,12/31 23:00:00,-5.1750,11.921481,12.029551,710174.145981,1.964846e+07,0.0,11.975516


In [17]:
idf_data['temp_diff'] =idf_data['t_in'] - idf_data['t_out']

In [18]:
idf_data['Date_Time'] = '2020/' + idf_data['Date_Time'].str.strip()
idf_data['Date_Time'] = idf_data['Date_Time'].str.replace('24:00:00','23:59:59')

In [19]:
idf_data['power_heating'] /= 3.6e6
idf_data['power_cooling'] /= 3.6e6
idf_data['power_electricity'] /= 3.6e6

In [20]:
idf_data['power_heating'] = idf_data['power_heating'].apply(lambda x: round(x, 2))
idf_data['power_cooling'] = idf_data['power_cooling'].apply(lambda x: round(x, 2))
idf_data['power_electricity'] = idf_data['power_electricity'].apply(lambda x: round(x, 2))
idf_data['t_in'] = idf_data['t_in'].apply(lambda x: round(x, 2))
idf_data['t_out'] = idf_data['t_out'].apply(lambda x: round(x, 2))

In [21]:
idf_data['total_power'] = idf_data['power_heating']+ idf_data['power_cooling'] #+idf_data['power_electricity']

In [22]:
idf_data = idf_data[['Date_Time','t_in','t_out','temp_diff','power_heating','power_cooling','power_electricity','total_power']]

In [23]:
idf_data.to_csv(path_or_buf='/home/ict4bd/Downloads/NA/Output/simulation_Data_2020.csv',index=False)